## This script computes the average histogram of a nifti images dataset and the histogram matching vs the computed average histogram

Author: Edda Boccia

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pandas as pd
import pydicom
import SimpleITK as sitk
import time
import nibabel as nib
from scipy import ndimage
from PIL import Image
from sklearn import preprocessing
%matplotlib inline

In [15]:
#modified: the original template image is represented here by t_values and t_counts(both computed in the main script)

import numpy as np

def hist_match(source, t_values,t_counts):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image

    Arguments:
    -----------
        source: np.ndarray
            Image to transform; the histogram is computed over the flattened
            array
        template: np.ndarray
            Template image; can have different dimensions to source
    Returns:
    -----------
        matched: np.ndarray
            The transformed output image
    """

    oldshape = source.shape
    source = source.ravel()
    #template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    print(bin_idx.shape)
    print(s_values.shape)
    print(s_counts.shape)
    #t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

In [16]:
path_orig = "path_to_N4BiasField/"  #images path after N4BF correction (each folder represents a patient, folder name = patientID)
path_Otsu = ""  #path where Otsu masks are stored
path_mask = ""  #output path 
patientID=[i.split('_')[0] for i in os.listdir(path_orig) if 'P_N' in i ]

In [ ]:

df_merged = pd.DataFrame()
# for cicle aiming at scaling all original images between 0 and 1 before computing their average histogram

for folder in patientID:
    
    Otsu_mask = np.load(path_orig+folder+'_P_OM.npy')       
    image = np.load(path_orig+folder+'_P_N4BF.npy')
    
    image_cropped = image * Otsu_mask
    image_T_1=image_cropped[image_cropped >0]     #exclude the background and keep ther body only
    image_T_2 = image_T_1.reshape(-1, 1)    #2-dimensional array where each subarray has 1 element
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaler.fit(image_T_2)
    image_T_2_scaled = scaler.transform(image_T_2)

    df1 = pd.DataFrame(image_T_2_scaled.ravel())
    df_merged = pd.concat([df_merged, df1], ignore_index=True)



In [20]:
#compute parameters used for histo matching
t_values_path = 'path_to_t/'

t_values = np.load(t_values_path + 't_values_HBP.npy') # Code for Hepatobiliary Phase, use PVP for Portal Venous Phase
t_counts_norm = np.load('t_counts_norm_HBP.npy')   #t_counts is normalized by the number of FPU patients

In [ ]:
#HISTO MATCHING (for each patient, matching is performed only inside the image body selected via Otsu masks)
#before histo matching, each original image is scaled between 0 and 1 (to be consistent with the average histo range)
#in MinMaxScaling, min and max are computed inside each original image body and then applied to the whole image

for folder in patientID:
    
    
        Otsu_mask = np.load(path_orig+folder+'_HBP_OM.npy')       
        image = np.load(path_orig+folder+'_HBP_N4BF.npy')


        image_cropped = image * Otsu_mask
        image_T_1=image_cropped[image_cropped >0]     #exclude the background and keep the body only
        min_value = image_T_1.min()
        max_value = image_T_1.max()

        image_cropped_01 = (image_cropped - min_value)/ (max_value - min_value)
        image_cropped_scaled = image_cropped_01 * Otsu_mask
    
        #perform histo matching only inside the body and replace the matched region in the original image
        loc = np.where(image_cropped_scaled != 0)
        pixels = image_cropped_scaled[loc]

        new_image = hist_match(pixels,t_values, t_counts_norm)

        image_cropped_scaled_matched = image_cropped_scaled.copy()
        for i, coord in enumerate(zip(loc[0], loc[1],loc[2])):
            image_cropped_scaled_matched[coord[0], coord[1],coord[2]] = new_image[i]
        print("fine")

        new_image_1=image_cropped_scaled_matched * Otsu_mask
        new_image_2 = new_image_1[new_image_1>0]

        image_path = os.path.join(path_mask,'Histo_matching_results_MinMaxScaling_noThresh',folder)
        
        img_np_test = nib.Nifti1Image(image_cropped_scaled_matched, affine=np.eye(4))
        nib.save(img_np_test, 'path_to_img_folder/'+folder+'_HBP_matched.nii.gz')

        